In [1]:
import pandas as pd
import numpy as np
from difflib import SequenceMatcher
from sentence_transformers import SentenceTransformer
import hdbscan
import umap
import warnings

# --- Global Configuration ---
warnings.simplefilter("ignore")

# --- Parameters for Cleaning --- 
INFRASTRUCTURE_LOG_PERCENT = 0.05
PREFIX_SUFFIX_N_CHARS = 10
SIMILARITY_THRESHOLD_HIGH = 0.90
OBSERVATION_STRING = "Nenhuma observação fornecida" # Portuguese for "No observation provided", replace with what's relevant for your dataset
NA_OBSERVATION_TARGET_PERCENT = 0.01

# --- Parameters for Frequency Balancing ---
DUPLICATE_LIMIT_TRIO = 3
DUPLICATE_LIMIT_PAIR = 6
DUPLICATE_LIMIT_DESC = 9
DUPLICATE_LIMIT_OBS = 9

# --- Parameters for 'Informative' Dataset ---
WORD_COUNT_THRESHOLD = 3
DISSIMILARITY_THRESHOLD = 0.6

# --- File Paths ---
INPUT_FILE = 'preliminary_cleaned_data.csv'
CLEANED_OUTPUT_FILE = 'updated_cleaned_data.csv'
INFORMATIVE_OUTPUT_FILE = 'informative_data.csv'

In [2]:
# =====================================================================================
# --- Helper Functions ---
# =====================================================================================

def initial_filter_and_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Performs initial filtering, component consolidation, and regex cleaning."""
    print("--- 2. Initial Filtering & Cleaning ---")
    df_copy = df.copy()

    # Limit infrastructure logs
    null_turbine_indices = df_copy.index[df_copy['Turbine No'].isnull()].tolist()
    if len(null_turbine_indices) > (df_copy.shape[0] * INFRASTRUCTURE_LOG_PERCENT):
        target_keep = int(df_copy.shape[0] * INFRASTRUCTURE_LOG_PERCENT)
        to_drop_count = len(null_turbine_indices) - target_keep
        indices_to_drop = np.random.choice(null_turbine_indices, size=to_drop_count, replace=False)
        df_copy.drop(indices_to_drop, inplace=True)

    # Filter for Component Name
    df_copy.dropna(subset=['Component Name'], inplace=True)
    df_copy = df_copy[df_copy['Component Name'] != '']
    
    # Consolidate Rotor Blade names
    component_name_map = {
        'Rotor Blade System 1': 'Rotor Blade System', 'Rotor Blade System 2': 'Rotor Blade System',
        'Rotor Blade System 3': 'Rotor Blade System', 'Rotor Blades Overall': 'Rotor Blade System'
    }
    df_copy['Component Name'] = df_copy['Component Name'].replace(component_name_map)

    # Regex cleaning
    wo_pattern = r'\(WO\s\d+\)'
    log_code_pattern = r'\b\d{2}\.\d{3}\.\d{2}-\d{4}\b'
    multiple_spaces_pattern = r'\s+'
    para_pattern = r'\s+para$'
    for col in ['Description', 'Observations']:
        df_copy[col] = (df_copy[col].astype(str)
                        .str.replace(wo_pattern, '', regex=True)
                        .str.replace(log_code_pattern, '', regex=True)
                        .str.replace(multiple_spaces_pattern, ' ', regex=True)
                        .str.strip())
    df_copy['Observations'] = df_copy['Observations'].str.replace(para_pattern, '', regex=True).str.strip()
    
    print("Initial filtering and cleaning complete.\n")
    return df_copy

def remove_redundancy(df: pd.DataFrame) -> pd.DataFrame:
    """Applies a cascade of filters to remove redundant rows."""
    print("--- 3. Removing Redundant Rows ---")
    df_copy = df.copy()

    # Exact matches
    df_copy = df_copy[df_copy['Description'] != df_copy['Observations']]

    # Prefix/Suffix matches
    def is_prefix_suffix_redundant(row, n):
        desc, obs = row['Description'], row['Observations']
        if not isinstance(desc, str) or not isinstance(obs, str): return False
        if obs.startswith(desc) and len(obs) <= len(desc) + n: return True
        if obs.endswith(desc) and len(obs) <= len(desc) + n: return True
        return False
    redundant_mask = df_copy.apply(is_prefix_suffix_redundant, n=PREFIX_SUFFIX_N_CHARS, axis=1)
    df_copy = df_copy[~redundant_mask]
    
    # High-similarity matches
    def calculate_similarity(a, b):
        if not isinstance(a, str) or not isinstance(b, str): return 0.0
        return SequenceMatcher(None, a, b).ratio()
    df_copy['similarity'] = df_copy.apply(lambda row: calculate_similarity(row['Description'], row['Observations']), axis=1)
    df_copy = df_copy[df_copy['similarity'] <= SIMILARITY_THRESHOLD_HIGH]
    df_copy.drop(columns=['similarity'], inplace=True)
    
    print("Redundancy removal complete.\n")
    return df_copy

def balance_frequencies(df: pd.DataFrame) -> pd.DataFrame:
    """Balances component and text frequencies through down-sampling and duplicate capping."""
    print("--- 4. Frequency Balancing & Final Content Filtering ---")
    df_copy = df.copy()

    # Down-sample the majority component class
    component_counts = df_copy['Component Name'].value_counts()
    if len(component_counts) > 1:
        majority_class_name = component_counts.index[0]
        cap_limit = component_counts.iloc[1]
        df_majority = df_copy[df_copy['Component Name'] == majority_class_name]
        df_others = df_copy[df_copy['Component Name'] != majority_class_name]
        if len(df_majority) > cap_limit:
            df_majority_downsampled = df_majority.sample(n=cap_limit, random_state=42)
            df_copy = pd.concat([df_majority_downsampled, df_others], ignore_index=True)

    # Filter out single-word observations
    df_copy = df_copy[df_copy['Observations'].str.split().str.len() != 1]

    # Cascading duplicate limits
    df_copy = df_copy[df_copy.groupby(['Component Name', 'Description', 'Observations']).cumcount() < DUPLICATE_LIMIT_TRIO]
    df_copy = df_copy[df_copy.groupby(['Description', 'Observations']).cumcount() < DUPLICATE_LIMIT_PAIR]
    df_copy = df_copy[df_copy.groupby('Description').cumcount() < DUPLICATE_LIMIT_DESC]
    df_copy = df_copy[df_copy.groupby('Observations').cumcount() < DUPLICATE_LIMIT_OBS]

    # Final down-sample of "No Observation" rows
    obs_rows_idx = df_copy.index[df_copy['Observations'] == OBSERVATION_STRING].tolist()
    if len(obs_rows_idx) > (df_copy.shape[0] * NA_OBSERVATION_TARGET_PERCENT):
        target_keep = int(df_copy.shape[0] * NA_OBSERVATION_TARGET_PERCENT)
        indices_to_drop = np.random.choice(obs_rows_idx, size=len(obs_rows_idx) - target_keep, replace=False)
        df_copy.drop(indices_to_drop, inplace=True)
        
    print(f"Frequency balancing complete. Shape of cleaned data before semantic step: {df_copy.shape}\n")
    return df_copy

def semantic_deduplication(df: pd.DataFrame) -> pd.DataFrame:
    """Performs advanced de-duplication using semantic clustering."""
    print("--- 5. Advanced Semantic De-duplication ---")
    if df.empty:
        print("Dataset is empty, skipping semantic de-duplication.\n")
        return df

    df_copy = df.copy()
    df_copy['combined_text'] = df_copy['Component Name'] + ' | ' + df_copy['Description'] + ' | ' + df_copy['Observations']
    
    print("Loading sentence embedding model...")
    model = SentenceTransformer('all-MiniLM-L6-v2')
    print(f"Generating semantic vectors for {len(df_copy)} logs... (This may take a moment)")
    embeddings = model.encode(df_copy['combined_text'].tolist(), show_progress_bar=True)
    
    print("Reducing vector dimensions with UMAP...")
    reducer = umap.UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=42)
    umap_embeddings = reducer.fit_transform(embeddings)
    
    print("Clustering logs with HDBSCAN...")
    clusterer = hdbscan.HDBSCAN(min_cluster_size=2, gen_min_span_tree=True)
    clusterer.fit(umap_embeddings)
    df_copy['cluster_id'] = clusterer.labels_
    
    initial_rows = df_copy.shape[0]
    df_copy = df_copy[df_copy.groupby('cluster_id').cumcount() < 3]
    print(f"Semantic de-duplication complete. Rows removed: {initial_rows - df_copy.shape[0]}")
    
    df_copy.drop(columns=['combined_text', 'cluster_id'], inplace=True)
    
    print(f"Shape of the final 'df_cleaned' dataset: {df_copy.shape}\n")
    return df_copy

def create_informative_dataset(df: pd.DataFrame) -> pd.DataFrame:
    """Creates a subset of the data with high-context, dissimilar text pairs."""
    print("--- 6. Creating High-Context 'df_informative' Dataset ---")
    df_info = df.copy()
    
    df_info = df_info[df_info['Observations'] != OBSERVATION_STRING]
    df_info = df_info[df_info['Description'].str.split().str.len() > WORD_COUNT_THRESHOLD]
    df_info = df_info[df_info['Observations'].str.split().str.len() > WORD_COUNT_THRESHOLD]
    
    def calculate_similarity(a, b):
        if not isinstance(a, str) or not isinstance(b, str): return 0.0
        return SequenceMatcher(None, a, b).ratio()
    df_info['similarity'] = df_info.apply(lambda row: calculate_similarity(row['Description'], row['Observations']), axis=1)
    df_info = df_info[df_info['similarity'] < DISSIMILARITY_THRESHOLD]
    df_info.drop(columns=['similarity'], inplace=True)
    
    print(f"Created 'df_informative' with {df_info.shape[0]} high-context rows.\n")
    return df_info

In [3]:
# =====================================================================================
# --- Main Execution Pipeline ---
# =====================================================================================

# 1. Load Data
print("--- 1. Loading Data ---")
try:
    df_raw = pd.read_csv(INPUT_FILE)
    print(f"Successfully loaded. Initial shape: {df_raw.shape}\n")
except FileNotFoundError:
    print(f"Error: Main data file not found at '{INPUT_FILE}'.\n")
    exit()

# 2. Run the full cleaning and balancing pipeline
df_cleaned = initial_filter_and_clean(df_raw)
df_cleaned = remove_redundancy(df_cleaned)
df_cleaned = balance_frequencies(df_cleaned)
df_cleaned = semantic_deduplication(df_cleaned)

# 3. Create the informative subset from the final cleaned data
df_informative = create_informative_dataset(df_cleaned)

# 4. Save final outputs
print("--- 7. Saving All Outputs ---")
df_cleaned.to_csv(CLEANED_OUTPUT_FILE, index=False)
print(f"Saved the large cleaned dataset to '{CLEANED_OUTPUT_FILE}'.")
df_informative.to_csv(INFORMATIVE_OUTPUT_FILE, index=False)
print(f"Saved the high-context dataset to '{INFORMATIVE_OUTPUT_FILE}'.")

print("\nProcess completed successfully.")

--- 1. Loading Data ---
Successfully loaded. Initial shape: (100, 13)

--- 2. Initial Filtering & Cleaning ---
Initial filtering and cleaning complete.

--- 3. Removing Redundant Rows ---
Redundancy removal complete.

--- 4. Frequency Balancing & Final Content Filtering ---
Frequency balancing complete. Shape of cleaned data before semantic step: (17, 13)

--- 5. Advanced Semantic De-duplication ---
Loading sentence embedding model...
Generating semantic vectors for 17 logs... (This may take a moment)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Reducing vector dimensions with UMAP...
Clustering logs with HDBSCAN...
Semantic de-duplication complete. Rows removed: 10
Shape of the final 'df_cleaned' dataset: (7, 13)

--- 6. Creating High-Context 'df_informative' Dataset ---
Created 'df_informative' with 1 high-context rows.

--- 7. Saving All Outputs ---
Saved the large cleaned dataset to 'updated_cleaned_data.csv'.
Saved the high-context dataset to 'informative_data.csv'.

Process completed successfully.
